# NOVAS PERGUNTAS DO CEO

# 0.0 Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import gridspec 
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.express as px
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
# pip install ipywidgets

In [ ]:
# pip install geopy

## 0.1. Loading Data

In [2]:
data = pd.read_csv('kc_house_data.csv')

###                 1.Qual a quantidade de imóveis por nível?
     - Nivel 0: Preço entre R$ 0.00 e R$ 321.950
     - Nível 1: Preço entre R$ 321.950 e R$ 450.00
     - Nível 2: Preço entre R$ 450.00 e R$ 645.00
     - Nível 3: Preço acima de R$ 645.00

In [3]:
data['nivel'] = 'NA'
for i in range(len(data)):
    if (data.loc[i, 'price']>0) & (data.loc[i,'price'] < 321950):
        data.loc[i, 'nivel'] = 'nivel_0'
            
    elif (data.loc[i, 'price'] >= 321950 ) & (data.loc[i, 'price'] < 450000):
        data.loc[i, 'nivel'] = 'nivel_1'
        
    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
        data.loc[i, 'nivel'] = 'nivel_2'
        
    else:
        data.loc[i, 'nivel'] = 'nivel_3'

### 2.Qual é a média do tamanho da sala de estar dos imóveis por 'size'?
     - Size 0: Tamanho entre  0 e  1427 sqft
     - Size 1: Tamanho entre  1427 e 1910 sqft
     - Size 2: Tamanho entre  1910 e 2550 sqft
     - Size 3: Tamanho acima de 2550 sqft

In [4]:
data['size'] = 'NA'
for i in range(len(data)):
    
    if (data.loc[i, 'sqft_living'] > 0) & (data.loc[i, 'sqft_living']<1427):
        data.loc[i, 'size'] = 'size_0'
    
    elif (data.loc[i, 'sqft_living'] >= 1427) & (data.loc[i, 'sqft_living']<1910):
        data.loc[i, 'size'] = 'size_1'
        
    elif (data.loc[i, 'sqft_living'] >= 1910) & (data.loc[i, 'sqft_living']<2550):
        data.loc[i, 'size'] = 'size_2'
        
    else:
        data.loc[i, 'size'] = 'size_3'        

### 3.Adicione as seguintes informações ao conjunto de dados originais:
    - Place Id: identificação da localização
    - OSM Type: Open Street Map type
    - Country: Nome do País
    - Country Code: Código do País

In [5]:
#from geopy.geocoders import Nominatim

## Create empty rows
#data['place_id'] = 'NA'
#data['osm_type'] = 'NA'
#data['country'] = 'NA'
#data['country_code'] = 'NA'

#geolocator = Nominatim(user_agent = 'geoapiExercises')

#for i in range(len(data)):
#    print('Loop:{} / {}'.format(i, len(data)))

## make query
#query = str(data.loc[i, 'lat']) +','+ str(data.loc[i, 'long'])
## API Request
#response = geolocator.reverse(query)

## Populate data

#if 'place_id' in response.raw['address']:
#    data.loc[i, 'place_id'] = response.raw['address']['place_id']
    
#if 'osm_type' in response.raw['address']:
#    data.loc[i, 'osm_type'] = response.raw['address']['osm_type']
    
#if 'country' in response.raw['address']:
#    data.loc[i, 'country'] = response.raw['address']['country']
    
#if 'contry_code' in response.raw['address']:
#    data.loc[i,'country_code'] = response.raw['address']['country_code']

In [6]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='geopyExercises')

query = '47.5112, -122.257'
response = geolocator.reverse(query)

In [7]:
import time 
from multiprocessing import Pool

In [8]:
data['query'] = data[['lat', 'long']].apply(lambda x: str(x['lat'])+','+str(x['long']), axis = 1)

In [9]:
def get_data(x):
    index, row = x
    time.sleep(1)
    
    #chamada API
    response = geolocator.reverse(row['query'])
    
    place_id = response.raw['place_id']
    osm_type = response.raw['osm_type']
    country = response.raw['address']['country']
    country_code = response.raw['address']['country_code']
    
    return place_id, osm_type, country, country_code

In [10]:
#import defs

#df1 = data[['id', 'query']].sample(10)

#p = Pool(3)

#start =time.process_time()
#df1[['place_id', 'osm_type', 'country', 'country_code']] = p.map(defs.get_data, df1.iterrows())
#end = time.process_time()

#print('Time Elapsed: {}', end-start)

### 4.Adicione os seguintes filtros no mapa:
    - Tamanho mínimo da área da sala de estar.
    - Número mínimo de banheiros.
    - Valor máximo do preço.
    - Tamanho máximo da área do porão.
    - Filtro das condições do imóvel.
    - Filtro por ano de construção.

In [11]:
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# define level of prices

data['level'] = data['price'].apply( lambda x: 0 if x <= 321950 
                                    else 1 if ( x > 321950 ) & ( x <= 450000)
                                    else 2 if ( x > 450000 ) & ( x <= 645000) else 3 )
data['level'] = data['level'].astype( int )

# Iteractive buttons

price_limit = widgets.IntSlider( 
    value = 540000,
    min = 75000,
    max = 7700000,
    step = 1,
    description='Maximun Price',
    disable=False,
    style={'description_width': 'initial'}
    )

waterfront_bar = widgets.Dropdown(
    options = data['is_waterfront'].unique().tolist(),
    value = 'yes',
    description = 'Water View',
    disable=False
    )

livingroom_limit = widgets.IntSlider( 
    value = int( data['sqft_living'].mean() ),
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step = 1,
    description='Minimum Living Room Size',
    disable=False,
    style={'description_width': 'initial'}
    )

bathroom_limit = widgets.IntSlider(
    value = int( data['bathrooms'].mean() ),
    min = data['bathrooms'].min(),
    max = data['bathrooms'].max(),
    step = 1,
    description='Minimum Bathroom Number',
    disable=False,
    style={'description_width': 'initial'}
    )
basement_limit = widgets.IntSlider(
    value = int( data['sqft_basement'].mean() ),
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step = 1,
    description='Minimum Basement Size',
    disable=False,
    style={'description_width': 'initial'}
    )
condition_limit = widgets.IntSlider(
    value = int( data['condition'].mean() ),
    min = data['condition'].min(),
    max = data['condition'].max(),
    step = 1,
    description='Minimum condition',
    disable=False,
    style={'description_width': 'initial'}
    )
year_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ),
    min = data['yr_built'].min(),
    max = data['yr_built'].max(),
    step = 1,
    description='Year Built',
    disable=False,
    style={'description_width': 'initial'}
    )


In [12]:
def update_map( df, waterfront, limit, livingroom_limit, bathroom_limit,basement_limit, condition_limit, year_limit ):
    houses = df[(df['price'] <= limit) &
            (df['is_waterfront'] == waterfront) &
            (df['sqft_living'] >= livingroom_limit) &
            (df['bathrooms'] >= bathroom_limit) &
            (df['sqft_basement'] >= basement_limit) &
            (df['condition'] >= condition_limit) &
            (df['yr_built'] >= year_limit )][['id', 'lat', 'long', 'price','level']]
        
    fig = px.scatter_mapbox( houses,
                             lat="lat",
                             lon="long",
                             size="price",
                             color="level",
                             color_continuous_scale=px.colors.cyclical.IceFire,
                             size_max=15,
                             zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()


In [13]:
widgets.interactive( update_map,
                    df=fixed( data ),
                    waterfront=waterfront_bar,
                    limit=price_limit,
                    livingroom_limit=livingroom_limit,
                    bathroom_limit=bathroom_limit,
                    basement_limit=basement_limit,
                    condition_limit=condition_limit,
                    year_limit=year_limit
                    )

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

### 5.Adicione os seguintes filtros no DashBoard:
    - Filtro por data disponivel para compra.
    - Filtro por ano de renovação.
    - Filtro se possui vista para a água ou não.

In [14]:
# Change data format
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

# Filtering

date_limit = widgets.SelectionSlider(
    options = data['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Max Available Date',
    disable = False,
    orientation = 'horizontal',
    style = {'description_width': 'initial'},
    readout = True
)

#Max year Renovated
year_limit = widgets.SelectionSlider(
    options = data['yr_renovated'].sort_values().unique().tolist(),
    value = 2000,
    description = 'Max Year',
    disable = False,
    continuous_update = False,
    orientation = 'horizontal',
    style = {'description_width': 'initial'},
    readout = True
)

# Waterfront
waterfront_limit = widgets.Checkbox(
    value =False,
    description = 'Waterfront?',
    disabled = False,
    indent = False
)

def update_map(data, date_limit, year_limit, waterfront_limit):
    #Filter data
    df = data[(data['date']<=date_limit) &
             (data['yr_renovated']>= year_limit) &
             (data['waterfront'] == waterfront_limit)].copy()

    fig = plt.figure(figsize = (24,12))
    specs = gridspec.GridSpec(ncols = 2, nrows = 2, figure = fig)
    
    ax1 = fig.add_subplot(specs[0,:]) # First Row
    ax2 = fig.add_subplot(specs[1,0]) # Second Row and First Column
    ax3 = fig.add_subplot(specs[1,1]) # Second Row and Second Column
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    sns.barplot(x ='year', y ='id', data = by_year, ax = ax1)
    
    # First Graph
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(x = 'date', y = 'id', data = by_day, ax = ax2)
    ax2.set_title('Title: Avg Price by Day')
    
    df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')
    by_week_of_year = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(x = 'year_week', y = 'id', data = by_week_of_year, ax = ax3)
    ax3.set_title('Title: Avg Price by week of Year')
    plt.xticks(rotation=60);
    

In [15]:
widgets.interactive(update_map,
                   data = fixed(data),
                   date_limit = date_limit,
                   year_limit = year_limit,
                   waterfront_limit = waterfront_limit)

interactive(children=(SelectionSlider(description='Max Available Date', index=212, options=('2014-05-02', '201…